The goal of this notebook is to find out the performance of the 5mo model when trying to predict the remaining 9mo. If it isn't very good, then the idea is to use this model to initialize a LwF model and use that to improve performance.

The conclusion that I came to is that there is no big difference in performance unless the initial training is only ~1000 points. In fact, fitting with 5mo and using the remaining 9mo for evaluation resulted in super nice performance, in fact better than fitting with all the 14mo data, which is weird. 

Anyway, in order to write a thesis I need a story and that is not a good story.

The stuff that really improves performance is `user_14`, so the goal will be to isolate that user in its own model and attribute performance improvements to 'continuous learning' and 'prototype models'.

Out of ~20K data points, 5.7K are `user_1` and ~12.1K are `user_14`. When fit alone, `user_14` has the best performance of all users (wmse 0.00395):

MTR-SWP & 0.128 & 0.00395 $\pm$ 0.00076 & 0.000160 $\pm$ 0.000087 \\

In [19]:
from importlib import reload
import numpy as np
import pandas as pd

import util
from types_ import DatasetConfiguration
import metrics
import estimators.linear as linear_estimators
import estimators.mtr as mtr_estimators
import plotting

from sklearn.model_selection import KFold
import metrics

### Load 5mo dataset and train
The index captures the first ~1500 points or so.

In [20]:
reload(util)
X, y, previous_recall_score, df = util.load_data(DatasetConfiguration(filename='logs-5mo.pkl'))
index = df['timestamp'] < 1590000000
#X, y, previous_recall_score, df = util.load_data()

Added 89186 logs
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneTyp

In [21]:

reload(metrics)
reload(mtr_estimators)
reload(linear_estimators)

SEEDS = [100,101]

estimator = None

def cross_validate_lomb_dataset(X, y, previous_recall_score, folds, seeds, estimator_constructor, estimator=None):    
    y_trues                = []
    y_preds                = []
    previous_recall_scores = []
    deltas                 = []

    for seed in seeds:
        kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
        for train_index, test_index in kf.split(X):

            X_train = X.iloc[train_index]
            y_train = y.iloc[train_index]
            X_test  = X.iloc[test_index]
            y_test  = y.iloc[test_index]
            delta_train = X['delta'].iloc[train_index]
            delta_test  = X['delta'].iloc[test_index]
            previous_recall_score_test = previous_recall_score.iloc[test_index]
            
            if estimator is None:
                estimator = estimator_constructor()
                estimator.fit(X_train, y_train, delta_train)

            y_pred = estimator.predict(X_test, delta_test)
            y_pred = np.clip(y_pred, 0.000001, 0.999999)

            y_trues.append(y_test)
            y_preds.append(y_pred)
            previous_recall_scores.append(previous_recall_score_test)
            deltas.append(delta_test)
    return metrics.LombEvaluation.from_CV(estimator.get_name(),
                                                y_trues, 
                                                y_preds, 
                                                previous_recall_scores, 
                                                deltas), estimator

for estimator_constructor in linear_estimators.all_estimators+mtr_estimators.all_estimators:
    evaluation, estimator = cross_validate_lomb_dataset(X[index], y[index], previous_recall_score[index], 5, SEEDS, estimator_constructor)
    print(evaluation.to_latex_tabular_row())

Linear Regression & 0.0962 & 0.0179 $\pm$ 0.0062 & 0.000518 $\pm$ 0.00065 \\
Logistic Regression & 0.0979 & 0.0209 $\pm$ 0.0072 & 0.000572 $\pm$ 0.00071 \\
HLR & 0.134 & 0.0167 $\pm$ 0.0093 & 0.000594 $\pm$ 0.00062 \\
HLR & 0.134 & 0.0168 $\pm$ 0.0093 & 0.000594 $\pm$ 0.00062 \\
MTR-Exp & 0.287 & 0.0114 $\pm$ 0.0035 & 0.000150 $\pm$ 0.00023 \\
MTR-ESq & 0.221 & 0.0125 $\pm$ 0.0036 & 0.000163 $\pm$ 0.00022 \\
MTR-Hyp & 0.288 & 0.0123 $\pm$ 0.0039 & 0.000150 $\pm$ 0.00024 \\
MTR-HSq & 0.217 & 0.0132 $\pm$ 0.0040 & 0.000162 $\pm$ 0.00023 \\
MTR-SWP & 0.151 & 0.0172 $\pm$ 0.0061 & 0.000165 $\pm$ 0.00022 \\


### Use the remaining nine months
To do this, first find out the largest timestamp of the `5mo` dataset and use it to filter out points which are in both datasets

In [22]:
timestamp_5mo_max = df['timestamp'].max()
estimator_ = estimator

In [23]:
X, y, previous_recall_score, df = util.load_data(DatasetConfiguration(filename='logs-14mo.pkl', users=['user_1']))

'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'tim

'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'tim

'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'tim

'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'tim

'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'tim

'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'timestamp'
'tim

Added 87118 logs
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneTyp

unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported op

unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
Index(['FIRST_EXPOSURE_seconds', 'user', 'timestamp',
       'TEXT__WORD_HIGHLIGHTED_amount', 'TEXT__SENTENCE_CLICK_amount',
       'TEXT__SENTENCE_READ_amount', 'TEXT__ALL_amount',
       'TEXT__WORD_HIGHLIGHTED_seconds', 'TEXT__SENTENCE_CLICK_seconds',
       'TEXT__SENTENCE_READ_seconds', 'TEXT__ALL_seconds',
       'REVISION__CLICKED_amount', 'REVISION__NO

Now the error rate is high :) This is what I wanted to motivate continuous learning and prototype models.

TODO: Store the estimators in an array and print out results for each estimator.

In [25]:
X_test = X[df['timestamp'] > timestamp_5mo_max]
y_test = y[df['timestamp'] > timestamp_5mo_max]
previous_recall_score_test = previous_recall_score[df['timestamp'] > timestamp_5mo_max]

evaluation, _ = cross_validate_lomb_dataset(X_test, y_test, previous_recall_score_test, 5, SEEDS, None, estimator_)
print(evaluation.to_latex_tabular_row())

MTR-SWP & 0.250 & 0.110 $\pm$ 0.13 & 0.0175 $\pm$ 0.045 \\
